In [32]:
import warnings
warnings.filterwarnings('ignore')

In [276]:
from pathlib import Path
from pandas.plotting import scatter_matrix

%matplotlib inline
import numpy as np
import datetime
import matplotlib as mpl 
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()
from sklearn.datasets import make_blobs
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn import preprocessing
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn import utils
pd.set_option('display.max_rows', 250)
pd.set_option('display.max_columns', 250)
pd.set_option('display.width', 150)

In [277]:
3
import os

import requests

import json


from dotenv import load_dotenv

import alpaca_trade_api as tradeapi

#from MCForecastTools import MCSimulation

import datetime

%matplotlib inline


# Load the environment variables from the .env file
#by calling the load_dotenv function

load_dotenv()

Python-dotenv could not parse statement starting at line 29
Python-dotenv could not parse statement starting at line 30


True

In [278]:
shareprice = us_shareprice_df = pd.read_csv(
    Path('us-shareprices-daily.csv'),
    index_col=['Ticker','Date'],
    parse_dates=True,
    infer_datetime_format=True,
    delimiter=';'
) 

shareprice.head()

SimFinId    Open     Low    High   Close  Adj. Close  Dividend  Volume  Shares Outstanding
Ticker Date                                                                                                  
GOOG   2014-03-27        18  568.00  552.92  568.00  558.46      558.46       NaN   13100         336050831.0
       2014-03-28        18  561.20  558.67  566.43  559.99      559.99       NaN   41100         336050831.0
       2014-03-31        18  566.89  556.93  567.00  556.97      556.97       NaN   10800         674462000.0
       2014-04-01        18  558.71  558.71  568.45  567.16      567.16       NaN    7900         674462000.0
       2014-04-02        18  565.11  562.19  604.83  567.00      567.00       NaN  146700         674462000.0

In [279]:
shareprice_gropu = shareprice.groupby(['Ticker', 'Date']).mean()
ticker_price = (shareprice_gropu[shareprice_gropu.index.get_level_values(level=1) > "2018-02-14"]) 

In [280]:
income = pd.read_csv('us-income-annual.csv',infer_datetime_format=True, parse_dates=True, delimiter=';')
income.rename(columns={'Report Date':'Date'}, inplace=True)

In [281]:
income.set_index(['Ticker', "Date"], inplace=True)

income.shape

(19255, 26)

In [282]:
balance = pd.read_csv('us-balance-annual.csv',infer_datetime_format=True, parse_dates=True, delimiter=';')
balance.rename(columns={'Report Date':'Date'}, inplace=True)

In [283]:
balance.set_index(['Ticker','Date'], inplace=True)

balance.shape

(19255, 28)

In [284]:
cash_flow = pd.read_csv('us-cashflow-annual.csv',infer_datetime_format=True, parse_dates=True, delimiter=';')
cash_flow.rename(columns={'Report Date':'Date'}, inplace=True)

In [285]:
cash_flow.set_index(['Ticker','Date'], inplace=True)

cash_flow.shape

(19238, 26)

In [300]:
balance_cash_flow = balance.merge(cash_flow, on=['Ticker', 'Date'])
income_balance_cash_flow = income.merge(balance_cash_flow, on=['Ticker', 'Date'])

income_balance_cash_flow.shape

(19238, 80)

In [301]:
data = income_balance_cash_flow.drop(columns=['SimFinId_x','Currency_x',
                                                    'Fiscal Year_x',
                                                    'Fiscal Period_x',
                                                    'Publish Date_x',
                                                    'Restated Date_x',
                                                    'Shares (Basic)_x',
                                                    'Shares (Diluted)_x'])

In [302]:
data.drop(columns=['SimFinId_y',
                   'Currency_y',
                   'Fiscal Year_y',
                   'Fiscal Period_y',
                   'Publish Date_y',
                   'Restated Date_y',
                   'Shares (Basic)_y',
                   'Shares (Diluted)_y'],
            inplace=True)

In [303]:
ticker_price = (shareprice_gropu[shareprice_gropu.index.get_level_values(level=1) > "2018-02-14"]) 

In [304]:
data_date = (data[data.index.get_level_values(level=1) > "2018-02-14"]) 


In [325]:
scaled_data_date = data_date.drop(columns=['Currency','Fiscal Year','Fiscal Period','Publish Date','Restated Date']) / 10000000

friday_skill = scaled_data_date.round(3)

friday_skill = friday_skill.fillna(0)
friday_skill

SimFinId  Shares (Basic)  Shares (Diluted)   Revenue  Cost of Revenue  Gross Profit  Operating Expenses  \
Ticker Date                                                                                                                  
A      2018-10-31     0.005          32.100            32.500   491.400         -222.700       268.700            -175.900   
       2019-10-31     0.005          31.400            31.800   516.300         -235.800       280.500            -186.400   
AA     2018-12-31     0.037          18.600            18.900  1340.300        -1008.100       332.200            -101.200   
       2019-12-31     0.037          18.500            18.500  1043.300         -853.700       189.600            -102.000   
AAL    2018-12-31     0.007          46.424            46.566  4454.100        -3265.100      1189.000            -923.400   
...                     ...             ...               ...       ...              ...           ...                 ...   
ZUO    2020-01-31     0.099          11.112            11.112    27.606          -13.418        14.188             -22.754   
ZYNE   2018-12-31     0.090           1.531             1.531     0.009            0.000         0.009              -4.048   
       2019-12-31     0.090           2.200             2.200     0.000            0.000         0.000              -3.432   
ZYXI   2018-12-31     0.017           3.250             3.404     3.192           -0.604         2.588              -1.551   
       2019-12-31     0.017           3.244             3.396     4.547           -0.881         3.666              -2.559   

                   Selling, General & Administrative  Research & Development  Depreciation & Amortization_x  Operating Income (Loss)  \
Ticker Date                                                                                                                            
A      2018-10-31                           -137.400                 -38.500                            0.0                   92.800   
       2019-10-31                           -146.000                 -40.400                            0.0                   94.100   
AA     2018-12-31                            -24.800                  -3.100                          -73.3                  231.000   
       2019-12-31                            -28.000                  -2.700                          -71.3                   87.600   
AAL    2018-12-31                           -152.000                   0.000                         -183.9                  265.600   
...                                              ...                     ...                            ...                      ...   
ZUO    2020-01-31                            -15.314                  -7.440                            0.0                   -8.566   
ZYNE   2018-12-31                             -1.324                  -2.725                            0.0                   -4.040   
       2019-12-31                             -1.394                  -2.038                            0.0                   -3.432   
ZYXI   2018-12-31                             -1.551                   0.000                            0.0                    1.037   
       2019-12-31                             -2.559                   0.000                            0.0                    1.107   

                   Non-Operating Income (Loss)  Interest Expense, Net  Pretax Income (Loss), Adj.  Abnormal Gains (Losses)  Pretax Income (Loss)  \
Ticker Date                                                                                                                                        
A      2018-10-31                        1.800                 -3.700                      94.600                    0.000                94.600   
       2019-10-31                       -2.200                 -3.800                      91.900                    0.000                91.900   
AA     2018-12-31  

In [395]:
sat_dad = friday_skill[['Revenue','Cost of Revenue','Operating Expenses']]
sat_dad['expenses'] = sat_dad['Cost of Revenue'] + sat_dad['Operating Expenses'] 

In [400]:
sat_dad['good_health'] = sat_dad['Revenue']  >= - sat_dad['expenses']
#X = scaled_data_date.drop(columns=['Revenue'])
#sat_dad['bad_health'] = (sat_dad['Revenue'] < sat_dad['health_check'])

In [401]:
X = sat_dad.drop(columns=['good_health'])

In [402]:
#y = friday_skill.reset_index(inplace=True)

In [403]:
y = sat_dad['good_health']
some_num = y.loc['ADVM':'AE']
some_num

Ticker  Date      
ADVM    2018-12-31    False
        2019-12-31    False
AE      2018-12-31     True
        2019-12-31     True
Name: good_health, dtype: bool

In [404]:
X_train, X_test, y_train, y_test  = train_test_split(X, y, test_size=0.20, random_state=42)

In [411]:
"""
Chapter 5: Support Vector Machines
A Support Vector Machine is a powerful versatile Machine Learning model, capable of performing linear&nonlinear class& regre.
SVM are well suited for classification of complex small or med datasets.

Linear SVM Classification 
You can think of SVM classifer as fitting the widest possible street between classes.
Called large margin classification.

Soft Margin Classification 

If we strictly impose that all instances must be off the street and on the right side,
This is called 'hard margin classification' . Two main issues with hard margins classification.
First it only works on data that is linearly separable.
Second  it is sensitive to outliers. 

To avoid these issues, use a more flexible model.
The objective is to find a good balance between keeping the street as large as possible\
limiting the the margin violations this is called 'soft marging classification'

When creating an SYM model using sklearn we can specify a number of hyperparameters. C is one of those HP.
Margin violations are bad usually better to have few of them.
"""

"\nChapter 5: Support Vector Machines\nA Support Vector Machine is a powerful versatile Machine Learning model, capable of performing linear&nonlinear class& regre.\nSVM are well suited for classification of complex small or med datasets.\n\nLinear SVM Classification \nYou can think of SVM classifer as fitting the widest possible street between classes.\nCalled large margin classification.\n\nSoft Margin Classification \n\nIf we strictly impose that all instances must be off the street and on the right side,\nThis is called 'hard margin classification' . Two main issues with hard margins classification.\nFirst it only works on data that is linearly separable.\nSecond  it is sensitive to outliers. \n\nTo avoid these issues, use a more flexible model.\nThe objective is to find a good balance between keeping the street as large as possiblelimiting the the margin violations this is called 'soft marging classification'\n\nWhen creating an SYM model using sklearn we can specify a number of h

In [405]:
# If your Large margin (left) is overfitting, you can try regularizing it by reducing C.


svm_clf = Pipeline([
    ('linear_svc', LinearSVC(C=1, loss='hinge'))
])

In [406]:
svm_clf.fit(X_train, y_train)

Pipeline(steps=[('linear_svc', LinearSVC(C=1, loss='hinge'))])

In [407]:
svm_clf.predict([some_num])
"""
Unlike Logistic Regression classifiers, SVM classifiers do not out probabilities each class

Instead of using the LinearSVC class we, could use the SVC  clas
with a linear kernel. When creating the SVC model, we would write SVC(kernl = "linear", c=1)
Or we could use the SGDClassifier class. With SGDClassifier(loss='hinge', alpha=1 (m*C))
^ applies regular Stochastic Gradient Descent to train a linear SVM classifier.
It does not converge as fast as the LinearSVC class but it can be useful to handle online classification tasks.
or huge datasets that do not fit in memory (out-of-core-training)
To center data subtract from it the mean.Or use standard scaler. Also set loss HP to 'hinge'} not default value.
For better performance set the dual HP to false, unless there are more features than training instances.

"""

array([ True])

In [412]:
"""
Linear SVM classifiers are efficient and work in many cases. 
Many datasets are not even close to being linearly separable.
One approach to handling nonlinear datasets is to add more features, such as polynominal features.
In some cases this can result in a linearly seprarable dataset. 



"""




polynomial_svm_clf = Pipeline([
    ("poly_features", PolynomialFeatures(degree=3)),
    ('scaler', StandardScaler()),
    ('svm_clf', LinearSVC(C=10, loss='hinge'))
])
polynomial_svm_clf.fit(X,y)
polynomial_svm_clf.predict([some_num])

array([ True])

In [420]:
"""
# Polynomial Kernel 
Adding Polynomial features is simple to implement and can work great with all sorts of ML algos
At a low Polynomial degree, this method cannot deal with very complex datasets.
With high Polynomial degree it creates a huge number of features, making the model too slow.

When using SVM you can apply an almost miraculous mathematical technique called thhe kernel trick 
The Kernl Trick makes it possible to get the same result as if you had added many polinomial features
even with very high-degree polynominals, without actually having to add them.
So there is no combinatorial explosion of the number features because you dont actually add any features.

"""

from sklearn.svm import SVC 
poly_kernel_svm_clf = Pipeline([
    ('scaler', StandardScaler()),
    ('svm_clf', SVC(kernel ='poly', degree=3, coef0=1,C=5))
])

poly_kernel_svm_clf.fit(X, y)

Pipeline(steps=[('scaler', StandardScaler()),
                ('svm_clf', SVC(C=5, coef0=1, kernel='poly'))])

In [422]:
poly_kernel_svm_clf.predict([some_num])

array([ True])

In [423]:
from sklearn.linear_model import SGDClassifier

In [424]:
sgd_clf = SGDClassifier(random_state=14)
"""
SGDClassifier depends pn randomness during training (hence "stochastic")
To reproduce results set randon_state parameter

"""
sgd_clf.fit(X_train, y_train)

SGDClassifier(random_state=14)

In [426]:
sgd_clf.predict([some_num])

array([ True])

In [427]:
from sklearn.model_selection import cross_val_score
"""
Cross_val_score() funtion to evalute our SGDClassifier model,
Using K-fold cross-validation with 3 folds
Remeber K-fold cross-validation means splitting the training set into K folds (In this case 3)
Making predictions and evaluating them on the test fold.
Then counts the number of correct predicions evalutaing them on each fold usitng a model trained on the remainding folds
"""

cross_val_score(sgd_clf, X_train, y_train, cv=3, scoring='accuracy')

array([0.92361111, 0.9632572 , 0.94438928])

In [428]:
"""
# Confusion Matrix 


The best way to to evaluate the performance of a classifier is look at the confusion matrix.
The idea is to count the number of times instances of a class A are classified as class B.

To compute the confusion  matrix, you need a set of predictions so that theu can be compared to the actual targets.
You can use the the cross_val_predict 
"""

from sklearn.model_selection import cross_val_predict

y_train_pred  = cross_val_predict(sgd_clf , X_train, y_train, cv=3)


In [429]:
"""
Like cross_val_score() func, sross_val_predict() K performs K-fold_cross-valdation instead of returning an evaluation scores, it returns the predictions made om each test fold

Ready tp get the the confusion matrix using the confusion_matrix(). Just pass the the target classes (y_train_5) and predicted class (y_train_pred)
"""
from sklearn.metrics import confusion_matrix

confusion_matrix(y_train, y_train_pred)

array([[ 668,   87],
       [  83, 2184]], dtype=int64)

In [430]:
"""
# Precision and Recall 

Scikit provides several functions to compute classifier metrics, including predsion and recall
"""

from sklearn.metrics import precision_score , recall_score
precision_score(y_train, y_train_pred)

0.9616908850726552

In [431]:
recall_score(y_train, y_train_pred)


0.9633877370974857

In [432]:
# To compute the F1 score, call the f1_score() fucntions

from sklearn.metrics import f1_score
f1_score(y_train, y_train_pred)


0.9625385632437198

In [433]:
y_scores = cross_val_predict(sgd_clf, X_train, y_train, cv=3,
                            method='decision_function')

In [434]:
"""
One way to compare classifiers is to measure the area uder the roc curve (AUC) 
a perfect classifer will have a ROC AUC equal to 1, where as a purely random classifier will have a ROC AUC equal to 0.5.
SKlearn provides a function to compute the ROC AUC
"""
from sklearn.metrics import roc_auc_score

roc_auc_score(y_train, y_scores)


0.9693740012912009

In [ ]:
"""
Similarity Features 
A technique to tackle nonlinear problems is to add features computed using a similarity function
Which measures how much each instance resembles a paticular landmark.



"""